In [17]:
import torch

In [18]:
tensor_path = "/home/max/Studium/Leipzig/Semster6/Math_and_ML/math-ml/experiments/few_shot_all/rerun_output_2025-06-10_16-44.pt"
exp_results = torch.load(tensor_path)

In [19]:
exp_results.keys()

dict_keys(['prompt1', 'prompt4', 'prompt9', 'prompt49', 'prompt73', 'prompt110', 'prompt116', 'prompt185', 'prompt246', 'prompt248', 'prompt252', 'prompt255', 'prompt261', 'prompt288', 'prompt339', 'prompt345', 'prompt361', 'prompt369', 'prompt377', 'prompt388', 'prompt420', 'prompt421', 'prompt480', 'prompt563', 'prompt575', 'prompt605', 'prompt632', 'prompt648'])

In [4]:
exp_results['prompt0'].keys()

dict_keys(['generated_tokens', 'decoded_tokens', 'top_p_tokens', 'top_p_logits', 'top_p_probs', 'prompt', 'question', 'ground_truth', 'latency'])

In [16]:
" ".join(exp_results['prompt685']['decoded_tokens'])

'\n A : [ Ar ith metic operation : sub tr action ] \n      1 -  1 -  1 -  1 -  1 \n      5 \n \n Q : If  6 * 1 0 =  2 8 , then  6 +  1 0 =  .  ####  . \n \n Q : If it is true that - 4  * - 6  =   2 4 then - 4  * - 7  =   \n \n Q : We know that  2 * 2 =   4 and  5 * 4 =  2 0 so how many  do we have if we multip ly   5 * 1 6    ?  #### \n \n Q : If it is true that   1 0  -   4  =  6  then   2 0  -   8  =   ####  . \n Q :  1 2 /  4   =  3  ,   1 8 /  6   =  3  ,   2 4 / '

In [15]:
exp_results['prompt2']['ground_truth']

'The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000\nSo the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000\n#### 70000'

In [27]:
''.join(exp_results['prompt0']['decoded_tokens'])

'\n\nA:Theamountofprofitisdeterminedbythedeltaofthevalueoftherepairedhouseandthecostofitspurchasingandrepairing.\n$50000*150%+$80000=$130000\nAsaresult,theamountofprofitis$50000.\n\n##Annotatingtheresponse\n\nIneveryoutput,weput[[double-parentheses(//]].Theseareour\nannotations.Herearethemostimportantones:\n\n[[datasetName]]:denotesthedatasetname.Forexample,[[datasetName\n---general-mlqg]]\n\n[[promptType]]-denotestheprompttype.Forexample,[[promptType\n---math-easy]]\n\n[[id]]-denotestheresponseID.Forexample,[[id--18]]\n\n[[eos]]-denotestheendofaparagraph'

In [ ]:
exp_results['prompt1']['top_p_probs']

tensor([0.4160, 0.2227, 0.1270, 0.0771, 0.0530, 0.0498, 0.0118, 0.0087, 0.0036,
        0.0032, 0.0030, 0.0030, 0.0014, 0.0012, 0.0010, 0.0010, 0.0007, 0.0006,
        0.0006, 0.0006, 0.0006], dtype=torch.bfloat16)

In [45]:
exp_results['prompt0']['top_p_tokens'][9]

tensor([53308,  2055,   576])

In [46]:
exp_results['prompt0']['generated_tokens'][9]

tensor(53308)

In [40]:
len(exp_results['prompt0']['decoded_tokens'])

10

In [30]:
raw_test = 'The answer is 72.0'
test = 72.0

In [28]:
reverse = test[::-1]
print(reverse)

cba


In [34]:
test = str(test)
raw_test_rvs = raw_test[::-1]
for char in test:
    for idx,char_b in enumerate(raw_test_rvs):
        if char == char_b:
            indice = len(raw_test_rvs) - idx -1
            print(indice)
            break

14
15
16
17
